In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy.optimize import brentq
import matplotlib.pylab as plt
from scipy.interpolate import interp1d
from scipy.optimize import least_squares
import matplotlib.ticker as ticker  
from scipy.integrate import quad

import sys
sys.path.append('..')

from analytical_option_formulae.option_types.vanilla_option import VanillaOption
vanilla_option = VanillaOption()

#please adjust this before running, its either SPX or SPY
filename = 'SPY_options'

In [2]:
#implied volatility reporting

def implied_volatility(S: float, K: float, r: float, price: float, T: float, options_type: str) -> float:
    try:
        bs_model = lambda x: vanilla_option.black_scholes_model(S, K, r, x, T)
        if (options_type.lower() == 'call'):
            implied_vol = brentq(lambda x: price -
                                bs_model(x).calculate_call_price(),
                                1e-12, 10.0)
        elif (options_type.lower() == 'put'):
            implied_vol = brentq(lambda x: price -
                                bs_model(x).calculate_put_price(),
                                1e-12, 10.0)
        else:
            raise NameError('Payoff type not recognized')
    except Exception:
        implied_vol = np.nan

    return implied_vol


In [3]:
#Read SPX data
df = pd.read_csv(f'{filename}.csv')
df['mid'] = 0.5*(df['best_bid'] + df['best_offer'])
df['strike'] = df['strike_price']*0.001
df['payoff'] = df['cp_flag'].map(lambda x: 'call' if x == 'C' else 'put')
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')
df['exdate'] = pd.to_datetime(df['exdate'], format='%Y%m%d')
df['days_to_expiry'] = (df['exdate'] - df['date']).dt.days
df['years_to_expiry'] = df['days_to_expiry']/365
#setup rates calculator
rates_df = pd.read_csv('zero_rates_20201201.csv')
rate_interpolate = interp1d(rates_df['days'], rates_df['rate'])
df['rates'] = rate_interpolate(df['days_to_expiry']) / 100.0 #make it in fractions so i dont forget

try:
    if filename.lower() == 'spy_options':
        S = 366.02
    elif filename.lower() == 'spx_options':
        S = 3662.45
    else:
        raise NameError('unknown input file')
except Exception as e:
    print(e)

#impl market volatility column
df['vols'] = df.apply(lambda x: implied_volatility(S, x['strike'], x['rates'], x['mid'], x['years_to_expiry'], x['payoff']),axis=1)
df.dropna(inplace=True)

#create market DF for each timestamp
days_to_expiry  = sorted(df['days_to_expiry'].unique())
summary_df = pd.DataFrame({'strike': df['strike'].unique() })
option_type = []
for days in days_to_expiry:
    day_df  = df[df['days_to_expiry'] == days]
    call_df = day_df[day_df['payoff'] == 'call']
    put_df  = day_df[day_df['payoff'] == 'put']
    strikes = sorted(day_df['strike'].unique())
    impliedvols = []
    for K in strikes:    
        if K > S:
            impliedvols.append(call_df[call_df['strike'] == K]['vols'].values[0])
            option_type.append('call')
        else:
            impliedvols.append(put_df[put_df['strike'] == K]['vols'].values[0])
            option_type.append('put')

    day_market_df = pd.DataFrame({'strike': strikes, days : impliedvols})
    summary_df = pd.merge(summary_df, day_market_df, how="outer", on='strike')

summary_df['option_type'] = summary_df.apply(lambda x: 'call' if x['strike'] > S else 'put', axis=1)
summary_df = summary_df.sort_values(by=['strike'])
summary_df = summary_df.reset_index()
summary_df = summary_df.drop(columns=['index'])
summary_df.to_csv(f'{filename}_vol_summary.csv')

#get atm implied vols, this is an approximation but good enough
atm_vols = {}
for i,days in enumerate(days_to_expiry):
    adjusted_summary_df = summary_df.iloc[:, [0,1+i,-1]].copy()
    adjusted_summary_df.dropna(inplace=True)    
    atm_vols[days] = np.interp(S,adjusted_summary_df.iloc[:, 0],adjusted_summary_df.iloc[:, 1])

In [4]:
start_date  = pd.to_datetime('2020-12-01')
end_date    = pd.to_datetime('2021-01-15')
day_diff = (end_date-start_date).days
T = day_diff/365.0
rate = rate_interpolate(day_diff)/100.0
sigma = atm_vols[day_diff]


Expected Black Scholes payoff is defined as
$$
E[V_T]= {S_0}^\frac{1}{3}e^{\frac{rT}{3}}e^{\frac{-\sigma^2T}{9}} + 1.5(log{S_0} + (r-\frac{\sigma^2}{2})T) + 10
$$

Therefore, the price is
$$
V_0 = e^{-rT}E[V_T]
$$


In [5]:
#black scholes payoff
def bs_price(S,rate,sigma,T):
    return np.exp(-rate*T) *((np.power(S,1.0/3.0) * np.exp((rate - 0.5 * sigma ** 2) * T * (1.0/3.0)) * np.exp(0.5 * (1.0/9.0) * T * sigma**2) \
                + 1.5* (np.log(S) + (rate - 0.5 * sigma ** 2) * T)  \
                + 10))


Expected Bachelier payoff defined as

$$
E[V_T] = \frac{1}{\sqrt{2\pi}}\int_{-\infty}^{\infty} (S_0 + \sigma S_0 x)^\frac{1}{3} e^\frac{-x^2}{2}\,dx +
\frac{1}{\sqrt{2\pi}}\int_{-\infty}^{\infty} 1.5log(S_0 + \sigma S_0 x) e^\frac{-x^2}{2}\,dx
+10
$$

Note  $$S_T = S_0 + \sigma S_0W_T$$

Therefore, the price is
$$
V_0 = e^{-rT}E[V_T]
$$


In [6]:
def integrand_1(x,S,sigma,T):
    return (1/np.sqrt(2*np.pi)) * np.power((S + sigma * S * np.sqrt(T) * x), 1.0/3.0) * np.exp(-0.5 * np.power(x,2))

def integrand_2(x,S,sigma,T):
    return (1/np.sqrt(2*np.pi)) * 1.5 * np.log(S + sigma * S * np.sqrt(T) * x) * np.exp(-0.5 * np.power(x,2))

def bachelier_price(S,rate,sigma,T):
    lower_bound = -1 / (sigma * np.sqrt(T)) #log term lower bound
    I_1 = quad(lambda x: integrand_1(x,S,sigma,T), lower_bound, np.inf)
    I_2 = quad(lambda x: integrand_2(x,S,sigma,T), lower_bound, np.inf)
    V_0_bachelier = np.exp(-rate*T) * (I_1[0] + I_2[0] + 10)
    return V_0_bachelier


In [7]:
spy_sabr_params = {17: {'alpha': 0.6654021927640178,
  'rho': -0.411899860796424,
  'nu': 5.249981412437772},
 45: {'alpha': 0.9081326347161665,
  'rho': -0.48877944853880195,
  'nu': 2.7285163433392956},
 80: {'alpha': 1.1209243554286754,
  'rho': -0.632939170293977,
  'nu': 1.742224770841638}}
spx_sabr_params = {17: {'alpha': 1.2122899548875727,
  'rho': -0.3009002722327193,
  'nu': 5.459761425517507},
 45: {'alpha': 1.8165044343707681,
  'rho': -0.40430176121217326,
  'nu': 2.7901583189643917},
 80: {'alpha': 2.140132609863112,
  'rho': -0.5749338828217627,
  'nu': 1.8417469754894316}}

try:
    if filename.lower() == 'spy_options':
        alpha = spy_sabr_params[day_diff]['alpha']
        rho = spy_sabr_params[day_diff]['rho']
        nu = spy_sabr_params[day_diff]['nu']
        beta = 0.7
    elif filename.lower() == 'spx_options':
        alpha = spx_sabr_params[day_diff]['alpha']
        rho = spx_sabr_params[day_diff]['rho']
        nu = spx_sabr_params[day_diff]['nu']
        beta = 0.7
    else:
        raise NameError('unknown input file')
except Exception as e:
    print(e)

#i just use prof Tee
def SABR(F, K, T, alpha, beta, rho, nu):
    X = K
    # if K is at-the-money-forward
    if abs(F - K) < 1e-12:
        numer1 = (((1 - beta)**2)/24)*alpha*alpha/(F**(2 - 2*beta))
        numer2 = 0.25*rho*beta*nu*alpha/(F**(1 - beta))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        VolAtm = alpha*(1 + (numer1 + numer2 + numer3)*T)/(F**(1-beta))
        sabrsigma = VolAtm
    else:
        z = (nu/alpha)*((F*X)**(0.5*(1-beta)))*np.log(F/X)
        zhi = np.log((((1 - 2*rho*z + z*z)**0.5) + z - rho)/(1 - rho))
        numer1 = (((1 - beta)**2)/24)*((alpha*alpha)/((F*X)**(1 - beta)))
        numer2 = 0.25*rho*beta*nu*alpha/((F*X)**((1 - beta)/2))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        numer = alpha*(1 + (numer1 + numer2 + numer3)*T)*z
        denom1 = ((1 - beta)**2/24)*(np.log(F/X))**2
        denom2 = (((1 - beta)**4)/1920)*((np.log(F/X))**4)
        denom = ((F*X)**((1 - beta)/2))*(1 + denom1 + denom2)*zhi
        sabrsigma = numer/denom

    return sabrsigma

For SABR payoff, we must retrieve the volatility using previous calibration result

$$
h(S_T) = S_T^{\frac{1}{3}} + 1.5 log(S_T) + 10 \\
h''(S_T) = -\frac{2}{9}S_T^{-\frac{5}{3}} - 1.5\frac{1}{S_T^2} \\
F = S_0 e ^ {rT}
$$

Therefore, the price is
$$ 
V_0 = e^{-rT}h(F) + \int_{0}^{F} h''(K)P(K) \,dK + \int_{F}^{\infty} h''(K)C(K) \,dK
$$

In [8]:
def h_func(F):
    return np.power(F,1.0/3.0) + 1.5 * np.log(F) + 10
def h_second_deriv(F):
    return (-2/9) * np.power(F,-5.0/3.0) - 1.5 * np.power(F,-2)

def h_put_integ(x,S,rate,T):
    #self, S: float, K: float, r: float, sigma: float, T: float
    F = S * np.exp(rate * T)
    sigma_p = SABR(F,x,T,alpha,beta,rho,nu)
    bsmodel = vanilla_option.black_scholes_model(S,x,rate,sigma_p,T)
    return h_second_deriv(x) * bsmodel.calculate_put_price()
    
def h_call_integ(x,S,rate,T):
    F = S * np.exp(rate * T)
    sigma_c = SABR(F,x,T,alpha,beta,rho,nu)
    bsmodel = vanilla_option.black_scholes_model(S,x,rate,sigma_c,T)
    return h_second_deriv(x) * bsmodel.calculate_call_price()

def sabr_price(S,rate, sigma, T):
    F = S * np.exp(rate * T)
    I_put = quad(lambda x: h_put_integ(x,S,rate,T), 1e-6, F)
    I_call = quad(lambda x: h_call_integ(x,S,rate,T), F, np.inf)
    V_0_SABR = np.exp(-rate*T) * h_func(F) +\
             I_put[0]   +   \
             I_call[0]
    return V_0_SABR


In [9]:
V_0_bachelier = bachelier_price(S,rate, sigma, T)
V_0_black_scholes = bs_price(S,rate,sigma,T)
V_0_SABR = sabr_price(S,rate,sigma,T)
print(f'{filename} for exotic 1 --> sigma = {sigma}; Black Scholes: {V_0_black_scholes:.7f}, Bachelier : {V_0_bachelier:.7f}, SABR: {V_0_SABR:.7f}')

SPX_options for exotic 1 --> sigma = 0.1849096526276905; Black Scholes: 37.7048975, Bachelier : 37.7031637, SABR: 37.7003687


# Model Free Integrated Variance - Black Scholes #

\begin{equation*}
\begin{split}
    \mathbb{E}\left[\int_0^T\sigma_t^2 \;dt\right] = 2e^{rT} \left(\int_0^{F}\frac{P(K)}{K^2}\;dK + \int_{F}^{\infty}\frac{C(K)}{K^2}\;dK\right)
    \end{split}
\end{equation*}



In [10]:
def bs_put_integ(x,S,rate,sigma,T):
    bsmodel = vanilla_option.black_scholes_model(S,x,rate,sigma,T)
    return bsmodel.calculate_put_price()/(np.power(x,2))
    
def bs_call_integ(x,S,rate,sigma,T):
    bsmodel = vanilla_option.black_scholes_model(S,x,rate,sigma,T)
    return bsmodel.calculate_call_price()/(np.power(x,2))

In [11]:
F = S * np.exp(rate*T)
I_put = quad(lambda x: bs_put_integ(x, S, rate, sigma, T), 0.0, F)
I_call = quad(lambda x: bs_call_integ(x, S, rate, sigma, T), F, np.inf)
bs_E_var = 2*np.exp(rate*T)*(I_put[0] + I_call[0])
bs_E_var

0.004215400228959317

# Model Free Integrated Variance - Bachelier #


In [12]:
def bach_put_integ(x,S,rate,sigma,T):
    bachmodel = vanilla_option.bachelier_model(S,x,rate,sigma,T)
    return bachmodel.calculate_put_price()/(np.power(x,2))
    
def bach_call_integ(x,S,rate,sigma,T):
    bachmodel = vanilla_option.bachelier_model(S,x,rate,sigma,T)
    return bachmodel.calculate_call_price()/(np.power(x,2))

In [13]:
F = S * np.exp(rate*T)
I_put = quad(lambda x: bach_put_integ(x, S, rate, sigma, T), 0.0, F)
I_call = quad(lambda x: bach_call_integ(x, S, rate, sigma, T), F, np.inf)
bach_E_var = 2*np.exp(rate*T)*(I_put[0] + I_call[0])
bach_E_var

0.004242501646917696

# Model Free Integrated Variance - SABR #


In [14]:
def sabr_put_integ(x,S,rate,T, alpha, beta, rho, nu):
    F = S * np.exp(rate * T)
    sigma_p = SABR(F,x,T,alpha,beta,rho,nu)
    bsmodel = vanilla_option.black_scholes_model(S,x,rate,sigma_p,T)
    return bsmodel.calculate_put_price()/(np.power(x,2))
    
def sabr_call_integ(x,S,rate,T, alpha, beta, rho, nu):
    F = S * np.exp(rate * T)
    sigma_c = SABR(F,x,T,alpha,beta,rho,nu)
    bsmodel = vanilla_option.black_scholes_model(S,x,rate,sigma_c,T)
    return bsmodel.calculate_call_price()/(np.power(x,2))


In [16]:
F = S * np.exp(rate*T)
I_put = quad(lambda x: sabr_put_integ(x, S, rate, T, alpha, beta, rho, nu), 0.0, F)
I_call = quad(lambda x: sabr_call_integ(x, S, rate, T, alpha, beta, rho, nu), F, np.inf)
sabr_E_var = 2*np.exp(rate*T)*(I_put[0] + I_call[0])
sabr_E_var

0.006350989023114547

In [17]:
print(f'{filename} for exotic 1 --> sigma = {sigma}; Black Scholes: {bs_E_var:.7f}, Bachelier : {bach_E_var:.7f}, SABR: {sabr_E_var:.7f}')

SPX_options for exotic 1 --> sigma = 0.1849096526276905; Black Scholes: 0.0042154, Bachelier : 0.0042425, SABR: 0.0063510
